In [1]:
import pandas as pd

In [2]:
test_actual_path = "data/EN_NEW/test.txt"
test_predicted_path = "results/EN_NEW_GCN_model_ELMo_H_combined_results_50/predicted_EN_NEW_model_ELMo_H_combined_system.txt"

In [3]:
with open(test_actual_path,'r') as f:
    test_actual = f.readlines()

In [4]:
with open(test_predicted_path,'r') as f:
    test_pred = f.readlines()

In [5]:
def sent_extractor(collection):
        sents = []
        tok_det = []
        for i in collection:
                if i=="\n":
                    sents.append(tok_det)
                    tok_det = []
                else:
                    tok_det.append(i)

        if len(tok_det) > 0:
            sents.append(tok_det)
        
        return sents

In [6]:
def read(sents):
		seqs = []
		for i in sents:
			seqs_i = []
			for j in i:
				j_list = j.split('\t')
				seqs_i.append([j_list[0], j_list[1], j_list[7]])
			seqs.append(seqs_i)
		
		return seqs

In [7]:
test_actual_sent = sent_extractor(test_actual) 
test_pred_sent = sent_extractor(test_pred)

In [8]:
test_actual_tags = read(test_actual_sent)

In [9]:
test_pred_tags = read(test_pred_sent)

In [10]:
def mwe_tag_index_extractor(tags):
    mwe_tags_start = {}
    for idx, sentence in enumerate(tags):
        print("Working on index {}".format(idx))
        mwe_tags_start[idx] = []
        for i,j in enumerate(sentence):
            mwe_indices = []
            if j[2] == 'B':
                print("------- Found B-tag at {}".format(i))
                k = i
                mwe_indices.append(k)
                if k+1 < len(sentence):
                    k += 1
                else:
                    break
                while sentence[k][2] == 'I':
                    print("---------------- Appended k: {}".format(k))
                    mwe_indices.append(k)
                    k += 1
                    if k >= len(sentence):
                        break
                mwe_tags_start[idx].append(mwe_indices)
    return mwe_tags_start

In [12]:
test_actual_mwe_tags = mwe_tag_index_extractor(test_actual_tags)

Working on index 0
------- Found B-tag at 13
---------------- Appended k: 14
------- Found B-tag at 17
---------------- Appended k: 18
------- Found B-tag at 28
Working on index 1
------- Found B-tag at 6
---------------- Appended k: 7
------- Found B-tag at 14
------- Found B-tag at 16
------- Found B-tag at 69
---------------- Appended k: 70
Working on index 2
------- Found B-tag at 2
---------------- Appended k: 3
---------------- Appended k: 4
---------------- Appended k: 5
------- Found B-tag at 7
---------------- Appended k: 8
---------------- Appended k: 9
------- Found B-tag at 14
---------------- Appended k: 15
---------------- Appended k: 16
------- Found B-tag at 17
---------------- Appended k: 18
------- Found B-tag at 24
---------------- Appended k: 25
------- Found B-tag at 30
---------------- Appended k: 31
------- Found B-tag at 40
---------------- Appended k: 41
---------------- Appended k: 42
Working on index 3
------- Found B-tag at 31
---------------- Appended k: 32

In [13]:
test_pred_mwe_tags = mwe_tag_index_extractor(test_pred_tags)

Working on index 0
------- Found B-tag at 17
------- Found B-tag at 39
Working on index 1
------- Found B-tag at 6
---------------- Appended k: 7
------- Found B-tag at 41
---------------- Appended k: 42
------- Found B-tag at 48
---------------- Appended k: 49
------- Found B-tag at 51
---------------- Appended k: 52
------- Found B-tag at 54
---------------- Appended k: 55
Working on index 2
------- Found B-tag at 2
---------------- Appended k: 3
---------------- Appended k: 4
------- Found B-tag at 7
---------------- Appended k: 8
---------------- Appended k: 9
------- Found B-tag at 14
------- Found B-tag at 17
---------------- Appended k: 18
------- Found B-tag at 24
---------------- Appended k: 25
------- Found B-tag at 90
---------------- Appended k: 91
Working on index 3
------- Found B-tag at 62
---------------- Appended k: 63
------- Found B-tag at 64
---------------- Appended k: 65
---------------- Appended k: 66
Working on index 4
------- Found B-tag at 2
---------------- A

In [15]:
sum([len(v) for k,v in test_actual_mwe_tags.items()]) 

1070

In [17]:
sum([len(v) for k,v in test_pred_mwe_tags.items()])

792

## True positive - Match found.
## True Negative - not in test set, not predicted
## False positive - Not in actual test set, but predicted
## False Negative - in the actual test set, but not predicted

In [50]:
def metrics(test_actual_mwe_tags, test_pred_mwe_tags):
    exact_matches_tp = 0
    exact_matches_fn = 0
    exact_matches_fp = 0
    tp_list = []
    fp_list = []
    fn_list = []
    for k,v in test_actual_mwe_tags.items():
        test_actual_tag_indices = v
        test_pred_tag_indices = test_pred_mwe_tags[k]
        print(k)
        print(test_actual_tag_indices)
        print(test_pred_tag_indices)
        exact_match_tp = 0
        exact_match_fn = 0
        exact_match_fp = 0
        for i in test_actual_tag_indices:
            print("Searching for a match for {}".format(i))
            if i in test_pred_tag_indices:
                print("True positive")
                exact_match_tp += 1
            else:
                print("True Negative")
                exact_match_fn += 1

        for i in test_pred_tag_indices:
            print("Searching for a match for {} in the actual test".format(i))
            if i not in test_actual_tag_indices:
                print("False positive")
                exact_match_fp += 1

        tp_list.append(exact_match_tp)
        fp_list.append(exact_match_fp)
        fn_list.append(exact_match_fn)

        exact_matches_tp += exact_match_tp
        exact_matches_fn += exact_match_fn
        exact_matches_fp += exact_match_fp
        
    return exact_matches_tp, exact_matches_fn, exact_matches_fp


In [43]:
tp, fn, fp = metrics(test_actual_mwe_tags, test_pred_mwe_tags)

0
[[13, 14], [17, 18], [28]]
[[17], [39]]
Searching for a match for [13, 14]
True Negative
Searching for a match for [17, 18]
True Negative
Searching for a match for [28]
True Negative
Searching for a match for [17] in the actual test
False positive
Searching for a match for [39] in the actual test
False positive
1
[[6, 7], [14], [16], [69, 70]]
[[6, 7], [41, 42], [48, 49], [51, 52], [54, 55]]
Searching for a match for [6, 7]
True positive
Searching for a match for [14]
True Negative
Searching for a match for [16]
True Negative
Searching for a match for [69, 70]
True Negative
Searching for a match for [6, 7] in the actual test
Searching for a match for [41, 42] in the actual test
False positive
Searching for a match for [48, 49] in the actual test
False positive
Searching for a match for [51, 52] in the actual test
False positive
Searching for a match for [54, 55] in the actual test
False positive
2
[[2, 3, 4, 5], [7, 8, 9], [14, 15, 16], [17, 18], [24, 25], [30, 31], [40, 41, 42]]
[[2

490
[[18, 19], [28, 29], [37, 38]]
[[18]]
Searching for a match for [18, 19]
True Negative
Searching for a match for [28, 29]
True Negative
Searching for a match for [37, 38]
True Negative
Searching for a match for [18] in the actual test
False positive
491
[[6, 7], [9, 10]]
[[6, 7], [8, 9, 10]]
Searching for a match for [6, 7]
True positive
Searching for a match for [9, 10]
True Negative
Searching for a match for [6, 7] in the actual test
Searching for a match for [8, 9, 10] in the actual test
False positive
492
[[14, 15]]
[[14, 15]]
Searching for a match for [14, 15]
True positive
Searching for a match for [14, 15] in the actual test
493
[[1, 2], [6, 7], [9, 10]]
[[1, 2]]
Searching for a match for [1, 2]
True positive
Searching for a match for [6, 7]
True Negative
Searching for a match for [9, 10]
True Negative
Searching for a match for [1, 2] in the actual test
494
[[2, 3]]
[]
Searching for a match for [2, 3]
True Negative
495
[[4, 5, 6], [13, 14, 15]]
[[4, 5, 6], [13, 14, 15]]
Sear

In [44]:
tp, fn, fp

(500, 570, 292)

In [45]:
rec = tp / (tp+fn)

In [46]:
pre = tp / (tp+fp)

In [47]:
rec, pre

(0.4672897196261682, 0.6313131313131313)

In [48]:
f1 = 2 / ((1.0/rec) + (1.0/pre))

In [49]:
f1

0.5370569280343717